
# Jet transport for the simple pendulum

In this notebook we will use `TaylorSeries.jl` and `TaylorIntegration.jl` in order to integrate the the simple pendulum, defined by the Hamiltonian:

$$
H(x, p) = \frac{1}{2}p^2-\cos x 
$$

The corresponding equations of motion are:

$$
    \begin{align}
        \dot x&=p \\
        \dot p&=-\sin x
    \end{align}
$$

We will integrate this problem for the initial condition $x(t_0)=x_0$, $p(t_0)=p_0$, as well as a neighbourhood $U_0$ around this initial condition, using the jet transport technique. For simplicity, we will take $p_0=0$, and $t_0=0$. Furthermore, we will choose $x_0$ such that the pendulum librates; that is, we will choose the numerical value of the energy, $E=H(x_0,p_0)=-\cos x_0$, such that the pendulum's motion in phase space is "below" (inside) the region bounded by the separatrix. Then, for the initial conditions given above, the librational period $T$ of the pendulum is

$$
T=\frac{4}{\sqrt{2}}\int_0^{x_0}\frac{dx}{\sqrt{\cos x_0-\cos x}}
$$

Or else, in terms of the complete elliptic integral of the first kind, $K$:

$$
T=4K(\sin(x_0/2))
$$

The jet transport technique consists in considering the neighbourhood $U_0$ as being parametrized by the sum $q_0+\xi$, where $q_0=(x_0,p_0)$ represents the coordinates of the initial condition in phase space, and $\xi=(\xi_1,\xi_2)$ represents an small variation with respect to this initial condition. We re-interpret each component of the sum $q_0+\xi$ as a multivariate polynomial in the variables $\xi_1$ and $\xi_2$, and then we integrate these multivariate polynomials in time using Taylor's method. 

The packages we need to load are:

In [16]:
Pkg.add("Elliptic")

INFO: Cloning cache of Elliptic from https://github.com/nolta/Elliptic.jl.git
INFO: Installing Elliptic v0.4.1
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of Elliptic
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [17]:
using TaylorIntegration, Plots, Elliptic

INFO: Precompiling module Elliptic.


The Hamiltonian for the simple pendulum is:

In [18]:
H(x) = 0.5x[2].^2-cos(x[1])

H (generic function with 1 method)

The LHS of the equations of motion is:

In [19]:
function pendulum!(t, x, dx)
    dx[1] = x[2] 
    dx[2] = -sin(x[1])
end

pendulum! (generic function with 1 method)

We must setup the the `TaylorN` variables necessary to perform the jet transport.; `varorder` represents the order of expansion in the variations $\xi$.

In [20]:
const varorder = 8
p = set_variables("ξ", numvars=2, order=varorder)

2-element Array{TaylorSeries.TaylorN{Float64},1}:
  1.0 ξ₁ + 𝒪(‖x‖⁹)
  1.0 ξ₂ + 𝒪(‖x‖⁹)

The nominal initial condition is:

In [21]:
q0 = [1.3, 0.0]

2-element Array{Float64,1}:
 1.3
 0.0

The initial value of the energy is:

In [22]:
H0 = H(q0)

-0.26749882862458735

The parametrization of the neighbourhood $U_0$ is represented by the following array of `TaylorN`'s:

In [23]:
q0TN = q0 + p

2-element Array{TaylorSeries.TaylorN{Float64},1}:
  1.3 + 1.0 ξ₁ + 𝒪(‖x‖⁹)
        1.0 ξ₂ + 𝒪(‖x‖⁹)

Evalute the Hamiltonian at $q0+\xi$:

In [24]:
H(q0TN)

 - 0.26749882862458735 + 0.963558185417193 ξ₁ + 0.13374941431229367 ξ₁² + 0.5 ξ₂² - 0.16059303090286547 ξ₁³ - 0.011145784526024473 ξ₁⁴ + 0.008029651545143275 ξ₁⁵ + 0.0003715261508674824 ξ₁⁶ - 0.00019118217964626843 ξ₁⁷ - 6.634395551205043e-6 ξ₁⁸ + 𝒪(‖x‖⁹)

Below, we setup some parameters for the Taylor integration. In particular, we will use a method of `taylorinteg` which returns the solution only at `t0`, `t0+integstep`, `t0+2integstep`,...,`tmax`, where `t0` and `tmax` are the initial and final times of integration, whereas `integstep` is the time interval between successive evaluations of the solution vector, chosen by the user. In this case, we are choosing `integstep` to be $\frac{1}{8}$th of the period of the pendulum.

In [25]:
const order = 28 #the order of the Taylor expansion wrt time
const abstol = 1e-20 #the absolute tolerance of the integration
const T = 4*Elliptic.K(sin(q0[1]/2)^2) #the librational period
const t0 = 0.0 #the initial time
const tmax = 6*T #the final time
const integstep = 0.125*T #the time interval between successive evaluations of the solution vector

0.8774062889805683

Then, we perform a Taylor integration over the initial condition `x0TN`:

In [26]:
#this is a "warmup lap"
@time xv = taylorinteg(pendulum!, q0TN, t0:integstep:tmax, order, abstol, maxsteps=1);

  1.041340 seconds (3.00 M allocations: 165.639 MiB, 7.17% gc time)


In [27]:
tv = t0:integstep:tmax;
@time xv = taylorinteg(pendulum!, q0TN, t0:integstep:tmax, order, abstol);

 55.218474 seconds (656.63 M allocations: 36.273 GiB, 23.91% gc time)


Note that we the integration above "works" for any initial neighborhood $U_0$ around the nominal initial condition $x_0$, of sufficiently small radius.

We will consider the particular case where $U_0$ is a disk of radius $r$, centered at $q_0$; that is $U_0=\{ q_0+\xi:\xi=(r\cos\phi,r\sin\phi); \phi\in[0,2\pi) \}$ for a given radius $r>0$. We will denote by $U_t$ the propagation of the initial neighbourhood $U_0$ evaluated at time $t$. Also, we denote by $q(t)$ the coordinates of the nominal solution at time $t$: $q(t)=(x(t),p(t))$. Likewise, we will denote the propagation at time $t$ of a given initial variation $\xi_0$ by $\xi(t)$. Then, we can compute the propagation of the boundary $\partial U_t$ of the neighbourhood $U_t$.

In [28]:
r = 0.05 #the radius of the neighbourhood
ϕ = 0.0:0.1:(2π+0.1) #the values of the angle
ξv1 = r*cos.(ϕ)
ξv2 = r*sin.(ϕ)
ξv = map(x->[r*cos(x), r*sin(x)], ϕ);

Evaluate the jet at $\partial U_{x(t)}$ (the boundary of $U_{x(t)}$) at each value of the solution vector `xv`. We organize these values such that we can plot them later:

In [29]:
@time xjet_plot = map(  ν->map(λ->map(μ->evaluate(μ, λ), ν), ξv), xv[:,1]  )
@time pjet_plot = map(  ν->map(λ->map(μ->evaluate(μ, λ), ν), ξv), xv[:,2]  );

  0.438005 seconds (342.91 k allocations: 18.104 MiB, 2.53% gc time)
  0.079041 seconds (55.30 k allocations: 2.827 MiB)


Evaluate the jet at the nominal solution, which corresponds to $\xi=(0,0)$, at each value of the solution vector `xv`:

In [30]:
x_nom = map(x->evaluate(x, [0.0, 0.0]), xv[:,1])
p_nom = map(x->evaluate(x, [0.0, 0.0]), xv[:,2]);

Now, we plot the nominal solution (black dots), as well as the evolution of the neighbourhood $U_0$ (in colors), each $\frac{1}{8}$th of a period. The initial condition corresponds to the black dot situated at $q_0=(1.3,0)$

In [31]:
plotly()
plot(
xjet_plot,
pjet_plot,
xaxis=("x",),
yaxis=("p",),
title="Simple pendulum phase space",
leg=false,
aspect_ratio=1.0
)
scatter!(
x_nom,
p_nom, 
color=:black,
m=(1,2.8,stroke(0))
)